In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import normalize
from tensorflow.keras import regularizers

2023-03-26 11:35:05.286784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 11:35:05.387007: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-26 11:35:05.389586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-26 11:35:05.389597: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
X_train = np.load('multichannel_dataset/X_train.npy')
X_test = np.load('multichannel_dataset/X_test.npy')
y_train = np.load('multichannel_dataset/y_train.npy')
y_test = np.load('multichannel_dataset/y_test.npy')

X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)
y_train = np.expand_dims(y_train, axis = -1)
y_test = np.expand_dims(y_test, axis = -1)

y_train = np.append(y_train, 1-y_train, axis = 1)
y_test = np.append(y_test, 1-y_test, axis = 1)

X_train = np.swapaxes(X_train, 1, 2)
X_test = np.swapaxes(X_test, 1, 2)

X_train.shape

(3561, 256, 6, 1)

In [22]:
input_shape = (256, 6, 1)

model = models.Sequential()
model.add(layers.Conv2D(40, (30, 1), kernel_regularizer=regularizers.L2(1e-2), activation="relu", padding="same", input_shape=input_shape))
model.add(layers.Conv2D(40, (1, 6), kernel_regularizer=regularizers.L2(1e-2), activation="relu", padding="valid"))
model.add(layers.AveragePooling2D((30, 1), strides=(15, 1)))
model.add((layers.Reshape((16,40))))
model.add(layers.GRU(40, return_sequences=True))

model.add(layers.Flatten())
model.add(layers.Dense(80, kernel_regularizer=regularizers.L2(1e-2), activation="relu"))
model.add(layers.Dense(2, activation="softmax"))

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 256, 6, 40)        1240      
                                                                 
 conv2d_32 (Conv2D)          (None, 256, 1, 40)        9640      
                                                                 
 average_pooling2d_9 (Averag  (None, 16, 1, 40)        0         
 ePooling2D)                                                     
                                                                 
 reshape_5 (Reshape)         (None, 16, 40)            0         
                                                                 
 gru_9 (GRU)                 (None, 16, 40)            9840      
                                                                 
 flatten_12 (Flatten)        (None, 640)               0         
                                                     

In [23]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])

history = model.fit(X_train, y_train, epochs=15, 
                    validation_data=(X_test, y_test))


Epoch 1/15
112/112 [==============================] - 3s 17ms/step - loss: 1.7587 - acc: 0.6332 - val_loss: 1.2736 - val_acc: 0.6510
Epoch 2/15
112/112 [==============================] - 2s 15ms/step - loss: 1.0524 - acc: 0.6872 - val_loss: 0.8993 - val_acc: 0.6801
Epoch 3/15
112/112 [==============================] - 2s 15ms/step - loss: 0.8262 - acc: 0.6832 - val_loss: 0.7493 - val_acc: 0.6813
Epoch 4/15
112/112 [==============================] - 2s 14ms/step - loss: 0.7205 - acc: 0.7009 - val_loss: 0.7128 - val_acc: 0.6756
Epoch 5/15
112/112 [==============================] - 2s 14ms/step - loss: 0.6637 - acc: 0.7018 - val_loss: 0.6478 - val_acc: 0.6835
Epoch 6/15
112/112 [==============================] - 2s 15ms/step - loss: 0.6354 - acc: 0.7071 - val_loss: 0.6244 - val_acc: 0.6925
Epoch 7/15
112/112 [==============================] - 2s 15ms/step - loss: 0.6130 - acc: 0.7012 - val_loss: 0.6123 - val_acc: 0.6824
Epoch 8/15
112/112 [==============================] - 2s 15ms/step - 

In [125]:
model.save('eeg_cnn')

INFO:tensorflow:Assets written to: eeg_cnn/assets


INFO:tensorflow:Assets written to: eeg_cnn/assets
